# Contextual embeddings in Python
Notebook created by A.Pournaki for the NLP course (ENS/Ecole nationale des chartes)
Revu par t. Poibeau (22 oct 2025)
`

In [1]:
## install new libraries
!pip install umap-learn
!pip install altair
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.6 MB/s  0:00:01m0:00:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 1.4 MB/s  0:00:27m0:00:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [umap-learn]5 [pynndescent]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 2.8 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [altair]s]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 2.6 MB/s  0:00:04m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 2.6 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 2.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 2.4 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 2.1 MB/s  0:00:35m0:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 1.8 MB/s  0:00:03 eta 0:00:

In [ ]:
## import
import umap
import pandas as pd
import altair as alt
from sentence_transformers import SentenceTransformer

In [ ]:
## load the articles
df = pd.read_csv("./data/articles_chatgpt.csv")
documents = list(df['text'])

In [ ]:
## load the multilingual model
## check out more models here:
## https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v1
model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

In [ ]:
## encode
document_vectors = model.encode(documents,show_progress_bar=True)

In [6]:
## let's inspect the shape of that array
document_vectors.shape

(493, 512)

In [7]:
## instantiate UMAP method
umap_embedding = umap.UMAP(n_components=2,
                           metric='euclidean')

In [8]:
## reduce dimensionality
document_vectors_2D = umap_embedding.fit_transform(document_vectors)

In [9]:
## add to the existing dataframe
df['X'] = document_vectors_2D[:,0]
df['Y'] = document_vectors_2D[:,1]

In [10]:
## plot
chart = alt.Chart(df).mark_circle(size=60).encode(
        alt.X('X'),
        alt.Y('Y'),
        tooltip=['title']
    ).properties(
        width=800,
        height=600,
        title='Article embedding'
    ).interactive()
chart.configure_title(
    fontSize=18,
    anchor='start',
    dx=20
)

alt.Chart(...)

### Exercice

📝 Do the same for the article text instead of titles. What changes?

📝 Do some clustering and display the clusters as colors in the interactive plot. Which method would you choose and why?


In [17]:
import umap
import numpy as np
import pandas as pd
import altair as alt

try:
    import hdbscan
    HDBSCAN_AVAILABLE = True
except Exception:
    HDBSCAN_AVAILABLE = False

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import umap

# ----- 2) Scale + UMAP
X_scaled = StandardScaler().fit_transform(document_vectors)
reducer = umap.UMAP(
    n_components=2,
    n_neighbors=15,       # raise for broader structure, lower for local detail
    min_dist=0.05,        # smaller -> tighter clusters in 2D
    metric='euclidean',
    random_state=None, n_jobs=-1  #42
)


emb = reducer.fit_transform(X_scaled)   # shape (n, 2)

# Recentre et met à l’échelle les coordonnées
emb_centered = emb - emb.mean(axis=0)
emb_scaled = emb_centered / np.abs(emb_centered).max(axis=0)


# ----- 3) Cluster on the UMAP embedding
if HDBSCAN_AVAILABLE:
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=25,   # tune: ↑ for bigger, fewer clusters; ↓ for smaller, more clusters
        min_samples=None,      # None -> equals min_cluster_size by default
        metric='euclidean',
        cluster_selection_epsilon=0.0
    ).fit(emb)
    labels = clusterer.labels_            # -1 = noise
    probs  = clusterer.probabilities_     # cluster membership strength [0..1]
    label_names = np.where(labels==-1, "noise", labels.astype(str))
else:
    # Fallback: k-means
    k = 4
    km = KMeans(n_clusters=k, n_init="auto", random_state=42)
    labels = km.fit_predict(emb)
    probs  = np.ones_like(labels, dtype=float)  # no probability in k-means
    label_names = labels.astype(str)

# ----- 4) DataFrame for plotting
df = pd.DataFrame({
    "x": emb_scaled[:,0],
    "y": emb_scaled[:,1],
    "cluster": label_names,
    "strength": probs
})

# Optional: make noise points semi-transparent in HDBSCAN
opacity = alt.condition(
    alt.datum.cluster == "noise",
    alt.value(0.25),
    alt.value(0.9)
) if HDBSCAN_AVAILABLE else alt.value(0.9)

# ----- 5) Altair interactive scatter
chart = alt.Chart(df).mark_circle(size=60).encode(
    x=alt.X("x", title="UMAP-1"),
    y=alt.Y("y", title="UMAP-2"),
    color=alt.Color("cluster:N", title="Cluster"),
    opacity=opacity,
    tooltip=["cluster", alt.Tooltip("strength:Q", format=".2f"), "x", "y"]
).properties(
    width=700, height=480, title="UMAP + clustering"
).interactive()

chart

alt.Chart(...)